In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

In [2]:
#get all the airplines' name
import requests
from bs4 import BeautifulSoup
url='https://www.airlinequality.com/review-pages/a-z-seat-reviews/'
page=requests.get(url)
soup=BeautifulSoup(page.content,'html.parser')
name_section=soup.find_all('div',class_='content')
names=[]
for letter in name_section:
    airlines=letter.find_all('li')
    for airline in airlines:
        name=airline.find('a').get_text(strip=True)
        names.append(name)
print(len(names))

201


In [3]:
#transform the name format
def name_transformation(name):
    import re
    name=name.lower()
    name=re.sub(' ','-',name)
    return name

In [4]:
url_name=[name_transformation(name) for name in names]
print(url_name)

['aegean-airlines', 'aer-lingus', 'aeroflot-russian-airlines', 'aerolineas-argentinas', 'aeromexico', 'air-astana', 'air-berlin', 'air-busan', 'air-canada', 'air-canada-rouge', 'air-china', 'air-corsica', 'air-dolomiti', 'air-europa', 'air-france', 'air-india', 'air-india-express', 'air-italy', 'air-mauritius', 'air-new-zealand', "air-north-yukon's-airline", 'air-serbia', 'air-tahiti-nui', 'air-transat', 'airasia', 'airasia-india', 'airasia-philippines', 'airasia-x', 'alaska-airlines', 'alitalia', 'allegiant-air', 'american-airlines', 'american-eagle', 'ana-all-nippon-airways', 'asiana-airlines', 'atlasglobal', 'austrian-airlines', 'avianca', 'azul-airlines', 'ba-cityflyer', 'bamboo-airways', 'bangkok-airways', 'batik-air', 'blue-air', 'boliviana-de-aviacion', 'british-airways', 'brussels-airlines', 'buddha-air', 'bulgaria-air', 'buta-airways', 'cathay-dragon', 'cathay-pacific-airways', 'cebu-pacific', 'china-airlines', 'china-eastern-airlines', 'china-southern-airlines', 'citilink', '

In [5]:
#get the page
def page_scraper(airline_name):
    url='https://www.airlinequality.com/seat-reviews/{}/?sortby=post_date%3ADesc&pagesize=100'
    try:
        response=requests.get(url.format(airline_name))
        soup=BeautifulSoup(response.content,'html.parser')
        panel=soup.find('article',class_='comp comp_reviews-pagination querylist-pagination position-')
        if panel==None:
            page=1
        else:
            page_list=panel.find_all('a')
            page=page_list[-2].text
    except requests.RequestException as e:
        print(f'Error fecting the page {airline_name}:{e}')
        page=None
    return page

In [6]:
seat_page={}
for airline in url_name:
    page=page_scraper(airline)
    seat_page[airline]=page
print(seat_page)

{'aegean-airlines': 1, 'aer-lingus': 1, 'aeroflot-russian-airlines': 1, 'aerolineas-argentinas': 1, 'aeromexico': 1, 'air-astana': 1, 'air-berlin': 1, 'air-busan': 1, 'air-canada': 1, 'air-canada-rouge': 1, 'air-china': 1, 'air-corsica': 1, 'air-dolomiti': 1, 'air-europa': 1, 'air-france': '2', 'air-india': 1, 'air-india-express': 1, 'air-italy': 1, 'air-mauritius': 1, 'air-new-zealand': 1, "air-north-yukon's-airline": 1, 'air-serbia': 1, 'air-tahiti-nui': 1, 'air-transat': 1, 'airasia': 1, 'airasia-india': 1, 'airasia-philippines': 1, 'airasia-x': 1, 'alaska-airlines': 1, 'alitalia': 1, 'allegiant-air': 1, 'american-airlines': '2', 'american-eagle': 1, 'ana-all-nippon-airways': 1, 'asiana-airlines': 1, 'atlasglobal': 1, 'austrian-airlines': 1, 'avianca': 1, 'azul-airlines': 1, 'ba-cityflyer': 1, 'bamboo-airways': 1, 'bangkok-airways': 1, 'batik-air': 1, 'blue-air': 1, 'boliviana-de-aviacion': 1, 'british-airways': '2', 'brussels-airlines': 1, 'buddha-air': 1, 'bulgaria-air': 1, 'buta-

In [7]:
def content_scraper(airline_name,max_page):
    url='https://www.airlinequality.com/seat-reviews/{}/page/{}/?sortby=post_date%3ADesc&pagesize=100'
    res=[]
    for i in range(1,int(max_page)+1):
        formatted_url=url.format(airline_name,i)
        response = requests.get(formatted_url)
        soup = BeautifulSoup(response.content, 'html.parser')
        review_list=soup.find_all('article', itemprop='review')
        for review in review_list:
            r={}
            r['title']=review.find('h2').get_text(strip=True) if review.find('h2') else 'No title'
            r['rating']=review.find('span',itemprop='ratingValue').get_text(strip=True) if review.find('span',itemprop='ratingValue') else 'No rating'
            customer_status=review.find('h3',class_='text_sub_header userStatusWrapper').get_text(strip=True)
            na_match=re.search(r'\(([^)]+)\)', customer_status)
            if na_match:
                r['nationality']=na_match.group(1)
            date_match=re.search(r'\d+\w*\s+[A-Za-z]+\s+\d{4}', customer_status)
            if date_match:
                r['date']=date_match.group(0)
            r['content']=review.find('div',class_='text_content').get_text(strip=True) if review.find('div',class_='text_content') else 'No content'
            for tr in review.find_all('tr'):
                detail_name = tr.find('td', class_='review-rating-header').text.strip()
                detail_value_container = tr.find('td', class_='review-value') or tr.find('td', class_='review-rating-stars stars')
                if detail_value_container:
                    stars = detail_value_container.find_all('span', class_='fill')
                    if stars:
                        detail_value = len(stars)
                    else:
                        detail_value = detail_value_container.text.strip()
                    r[detail_name] = detail_value
            res.append(r)
    return res

In [8]:
final_seats_data={}
for seat_name,seat_page in seat_page.items():
    res=content_scraper(seat_name, seat_page)
    seat_df=pd.DataFrame(res)
    seat_df['Seat']=seat_name
    final_seats_data[seat_name]=seat_df
print(len(final_seats_data))

201


In [9]:
seat_dt=pd.concat(final_seats_data.values(),ignore_index=True)
seat_dt.head()

,title,rating,nationality,date,content,Seat Type,Aircraft Type,Seat Layout,Date Flown,Type Of Traveller,...,Power Supply,Seat Storage,Recommended,Sleep Comfort,Sitting Comfort,Seat/bed Width,Seat/bed Length,Seat Privacy,Viewing Tv Screen,Seat
0,"""The seat pitch was awful""",3,Belgium,13th April 2024,✅Trip Verified| The new A320neo feels worse t...,Economy Class,A320neo,3x3,April 2024,Couple Leisure,...,5.0,4.0,no,NaN,NaN,NaN,NaN,NaN,NaN,aegean-airlines
1,"""the new cabin is a downgrade""",2,Germany,7th November 2023,✅Trip Verified| Took a flight at 25B on A320n...,Economy Class,A320neo,3x3,October 2023,Couple Leisure,...,2.0,1.0,no,NaN,NaN,NaN,NaN,NaN,NaN,aegean-airlines
2,"""middle seat was kept empty""",5,United Kingdom,23rd October 2021,✅Trip Verified| Seating seemed to be the same...,Business Class,A320,2x2,October 2021,Leisure,...,NaN,3.0,yes,3.0,3.0,2.0,3.0,2.0,NaN,aegean-airlines
3,"""have to pay for extra leg room""",4,Greece,25th February 2020,"✅Trip Verified| Very good overall, but I'm ta...",Economy Class,A320,3x3,February 2020,Business,...,3.0,3.0,yes,NaN,NaN,NaN,NaN,NaN,3.0,aegean-airlines
4,"""Never had less legroom""",1,Switzerland,21st February 2020,✅Trip Verified| Never had less legroom on a f...,Economy Class,A320neo,3x3,February 2020,Business,...,5.0,1.0,no,NaN,NaN,NaN,NaN,NaN,NaN,aegean-airlines


In [10]:
seat_dt.columns

Index(['title', 'rating', 'nationality', 'date', 'content', 'Seat Type',
       'Aircraft Type', 'Seat Layout', 'Date Flown', 'Type Of Traveller',
       'Seat Legroom', 'Seat Recline', 'Seat Width', 'Aisle Space',
       'Power Supply', 'Seat Storage', 'Recommended', 'Sleep Comfort',
       'Sitting Comfort', 'Seat/bed Width', 'Seat/bed Length', 'Seat Privacy',
       'Viewing Tv Screen', 'Seat'],
      dtype='object')

In [11]:
raw_seat_dt=seat_dt.loc[:,['Seat','title', 'rating', 'nationality', 'date', 'content', 'Seat Type',
       'Aircraft Type', 'Seat Layout', 'Date Flown', 'Type Of Traveller',
       'Seat Legroom', 'Seat Recline', 'Seat Width', 'Aisle Space',
       'Power Supply', 'Seat Storage', 'Recommended', 'Sleep Comfort',
       'Sitting Comfort', 'Seat/bed Width', 'Seat/bed Length', 'Seat Privacy',
       'Viewing Tv Screen']]
raw_seat_dt.head()

,Seat,title,rating,nationality,date,content,Seat Type,Aircraft Type,Seat Layout,Date Flown,...,Aisle Space,Power Supply,Seat Storage,Recommended,Sleep Comfort,Sitting Comfort,Seat/bed Width,Seat/bed Length,Seat Privacy,Viewing Tv Screen
0,aegean-airlines,"""The seat pitch was awful""",3,Belgium,13th April 2024,✅Trip Verified| The new A320neo feels worse t...,Economy Class,A320neo,3x3,April 2024,...,3.0,5.0,4.0,no,NaN,NaN,NaN,NaN,NaN,NaN
1,aegean-airlines,"""the new cabin is a downgrade""",2,Germany,7th November 2023,✅Trip Verified| Took a flight at 25B on A320n...,Economy Class,A320neo,3x3,October 2023,...,1.0,2.0,1.0,no,NaN,NaN,NaN,NaN,NaN,NaN
2,aegean-airlines,"""middle seat was kept empty""",5,United Kingdom,23rd October 2021,✅Trip Verified| Seating seemed to be the same...,Business Class,A320,2x2,October 2021,...,NaN,NaN,3.0,yes,3.0,3.0,2.0,3.0,2.0,NaN
3,aegean-airlines,"""have to pay for extra leg room""",4,Greece,25th February 2020,"✅Trip Verified| Very good overall, but I'm ta...",Economy Class,A320,3x3,February 2020,...,3.0,3.0,3.0,yes,NaN,NaN,NaN,NaN,NaN,3.0
4,aegean-airlines,"""Never had less legroom""",1,Switzerland,21st February 2020,✅Trip Verified| Never had less legroom on a f...,Economy Class,A320neo,3x3,February 2020,...,3.0,5.0,1.0,no,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
raw_seat_dt.to_csv('seat_data.csv', index=False)